In [1]:
import os
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras import layers
import tensorflow.keras as keras
from PIL import Image
import numpy as np

In [2]:
src_dir = "../datasets/ori/"
train_dir = "../datasets/train/"
val_dir = "../datasets/validation/"
test_dir = "../datasets/test/"
categories = ['igneous/', 'metamorphic/', 'sedimentary/']
for category in categories:
    file_list = os.listdir(src_dir+category)
    for n, i in enumerate(file_list[:200]):
        img = Image.open(src_dir+category+i)
        img_resize = img.resize((150, 150))
        img_resize.save(train_dir+category[:-1]+"_"+str(n)+".jpg", "JPEG")
    for n, i in enumerate(file_list[200:300]):
        img = Image.open(src_dir+category+i)
        img_resize = img.resize((150, 150))
        img_resize.save(val_dir+category[:-1]+"_"+str(n)+".jpg", "JPEG")
    for n, i in enumerate(file_list[300:400]):
        img = Image.open(src_dir+category+i)
        img_resize = img.resize((150, 150))
        img_resize.save(test_dir+category[:-1]+"_"+str(n)+".jpg", "JPEG")

In [3]:
train_data = np.empty(shape=(1, 150, 150, 3), dtype=np.float64)
for i in os.listdir(train_dir):
    img = Image.open(train_dir+i)
    train_data = np.append(train_data, [np.asarray(img)], axis=0)
train_data = np.delete(train_data, [0,0], axis=0)
train_data = train_data / 255.0

In [4]:
a = [0 for i in range(200)] + [1 for j in range(200)] + [2 for k in range(200)]
train_labels = np.array(a)

In [5]:
s = np.arange(600)
np.random.shuffle(s)
train_data = train_data[s]
train_labels = train_labels[s]
print(train_data.shape, train_labels.shape)

(600, 150, 150, 3) (600,)


In [6]:
# with tf.device('/CPU:0'):
#     inputs = keras.Input(shape=(150, 150, 3,))
#     flat = layers.Flatten()(inputs)
#     fc1 = layers.Dense(64, activation='relu')(flat)
#     fc2 = layers.Dense(128, activation='relu')(fc1)
#     fc3 = layers.Dense(256, activation='relu')(fc2)
#     output = layers.Dense(3, activation='softmax')(fc3)
#     model = Model(inputs=inputs, outputs=output, name="FC_Model")
#     model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
# model.summary()

In [7]:
# with tf.device('/CPU:0'):
#     model.fit(train_data, train_labels, epochs=100)

In [8]:
# with tf.device('/CPU:0'):
#     inputs = keras.Input(shape=(150, 150, 3,))
#     conv1 = layers.Conv2D(64, (3, 3), activation='relu')(inputs)
#     pool1 = layers.MaxPool2D((2, 2))(conv1)
#     drop1 = layers.Dropout(0.4)(pool1)
#     conv2 = layers.Conv2D(128, (3, 3), activation='relu')(drop1)
#     pool2 = layers.MaxPool2D((2, 2))(conv2)
#     drop2 = layers.Dropout(0.4)(pool2)
#     conv3 = layers.Conv2D(256, (3, 3), activation='relu')(drop2)
#     flat = layers.Flatten()(conv3)
#     output = layers.Dense(3, activation='softmax')(flat)
#     model = Model(inputs=inputs, outputs=output, name="CNN")
#     model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
# model.summary()

In [9]:
# model.fit(train_data, train_labels, epochs=30, validation_split=0.2)

In [10]:
base_model = keras.applications.VGG16(input_shape=(150, 150, 3),
                                      include_top=False,
                                      weights='imagenet')
base_model.trainable = False
base_model.summary()

2021-12-17 16:00:14.170883: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-17 16:00:14.171075: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-17 16:00:14.196175: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-17 16:00:14.196368: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-17 16:00:14.196652: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from S

58900480/58889256 [==============================] - 3s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                              

In [11]:
top_model = keras.models.Sequential()
top_model.add(layers.Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(layers.Dense(256, activation='relu'))
top_model.add(layers.Dense(512, activation='relu'))
top_model.add(layers.Dense(3, activation='softmax'))

top_model.load_weights(top_model)

top_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 256)               2097408   
                                                                 
 dense_1 (Dense)             (None, 512)               131584    
                                                                 
 dense_2 (Dense)             (None, 3)                 1539      
                                                                 
Total params: 2,230,531
Trainable params: 2,230,531
Non-trainable params: 0
_________________________________________________________________


In [13]:
from tensorflow.keras import applications
model = applications.VGG16(weights='imagenet', input_shape=(150, 150, 3))
top_model = keras.Sequential()
top_model.add(layers.Flatten(input_shape=model.output_shape[1:]))
top_model.add(layers.Dense(256, activation='relu'))
top_model.add(layers.Dense(2, activation='softmax'))

ValueError: When setting `include_top=True` and loading `imagenet` weights, `input_shape` should be (224, 224, 3).  Received: input_shape=(150, 150, 3)